### Load data

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [5]:
%cd gdrive/My\ Drive/Colab Notebooks/data/

/content/gdrive/My Drive/Colab Notebooks/data


In [6]:
!ls

 evgenyi_onegin.txt   model.hdf5	        model.txt   train.csv  'отзывы за лето.xls'
 model.bin	      model.model.vectors.npy   test.csv    val.csv


In [7]:
path_to_file = 'evgenyi_onegin.txt'

In [8]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [9]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [10]:
text = text + text

In [11]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [12]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int, text, len(text_as_int), len(text)

### Train and Target

In [14]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [15]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [16]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [17]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [20]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [21]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()

        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')

        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16291331 (62.15 MB)
Trainable params: 16291331 (62.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 6.5489335e-04, -4.8312504e-04, -1.1105703e-03, ...,
        -1.6380972e-04, -1.5179185e-03,  4.5000357e-04],
       [-5.3335563e-05, -1.0737600e-03, -1.5385547e-03, ...,
        -1.5652558e-03, -3.0320077e-03,  6.4675382e-04],
       [ 3.0620300e-05, -7.6243293e-04, -1.2437438e-03, ...,
        -2.3714507e-03, -2.6710490e-03,  3.2641389e-04],
       ...,
       [-3.0290801e-04, -7.6642111e-03, -5.7911766e-03, ...,
        -1.4378047e-03,  1.9030582e-02,  8.3983783e-03],
       [-4.6601053e-04, -7.6678745e-03, -5.9802402e-03, ...,
        -1.2579074e-03,  1.9307733e-02,  8.5491054e-03],
       [-5.8460608e-04, -7.6673673e-03, -6.1174468e-03, ...,
        -1.1349642e-03,  1.9478992e-02,  8.6575691e-03]], dtype=float32)>

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'скать?\n\n                                                                   Дмитриев.\n\n              '

Next Char Predictions: 
 'Г3яilЮкЖ9kЖSЭУhFХоh?и,)4?0GqsX"Г uz.;бdЦz6жНqEЕд-0ц1tо!ХГеyоo4тIьЬc-зЕуPFГССзkю?yьтЦPТИ}ОТнзmЯ1.АHМl'


### Train the model

In [30]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8781433


In [31]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [32]:
!rm -rf ./training_checkpoints

In [33]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [35]:
EPOCHS = 20

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 25s 203ms/step - loss: 1.9584
Epoch 2/20
88/88 [==============================] - 19s 201ms/step - loss: 1.4366
Epoch 3/20
88/88 [==============================] - 21s 212ms/step - loss: 1.2911
Epoch 4/20
88/88 [==============================] - 20s 216ms/step - loss: 1.2055
Epoch 5/20
88/88 [==============================] - 21s 223ms/step - loss: 1.1167
Epoch 6/20
88/88 [==============================] - 20s 220ms/step - loss: 1.0301
Epoch 7/20
88/88 [==============================] - 21s 227ms/step - loss: 0.9411
Epoch 8/20
88/88 [==============================] - 21s 222ms/step - loss: 0.8672
Epoch 9/20
88/88 [==============================] - 21s 227ms/step - loss: 0.7785
Epoch 10/20
88/88 [==============================] - 20s 212ms/step - loss: 0.6931
Epoch 11/20
88/88 [==============================] - 20s 219ms/step - loss: 0.6161
Epoch 12/20
88/88 [==============================] - 21s 228ms/step - loss: 0.5070
Epoch 13/20
8

## Generate text

In [37]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_18'

In [38]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_3 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16291331 (62.15 MB)
Trainable params: 16291331 (62.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [41]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже воро                                                                                                                                                                                                                                                                                                                                                                                                     ра                                                                                                         


In [42]:
len(text_)

515